In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
gender = pd.read_csv("gender.csv")
gender = gender.drop(['Unnamed: 0', 'School'], axis = 1)

eth = pd.read_csv("race.csv")
eth = eth.drop(['Unnamed: 0', 'School'], axis = 1).rename(columns = {'Uad Uc Ethn 6 Cat': 'Ethnicity'})

In [ ]:
gender

In [ ]:
# features: proportion of applicants that were male
# proportion admitted male

# target: in state or not AND in state, out of state, international

In [ ]:
# we need schools with both numbers for # of students applied and # of students admitted
# so we need to do some wranging to get that

In [ ]:
gender_filtered = gender.groupby('Calculation1').filter(lambda df: len(df.loc[df['Count'] == 'Adm', 'Count']) > 0)

In [ ]:
gender_filtered

In [ ]:
# not all schools have numbers for all categories – we need to interpolate them for consistency

In [ ]:
all_count, male_count, female_count = 0, 0, 0

In [ ]:
for name, df in gender_filtered.groupby('Calculation1'):
    app = df.loc[df['Count'] == 'App', 'Gender']
    app = sorted(app.tolist())
    
    if app == ['All']:
        # No gender data
        pass
    
    if app == sorted(['All', 'Female', 'Male']):
#         print('has all')
        all_count += 1

    if app == ['All', 'Male']:
#         print(name)
        male_count += 1
    
    if app == ['All', 'Female']:
        female_count += 1
    
#     print(df)

In [ ]:
# jk – the majority of schools do have all of that data, let's just work with them

In [ ]:
gender_filtered = gender_filtered.groupby('Calculation1').filter(
    lambda df: (sorted(df.loc[df['Count'] == 'App', 'Gender'].tolist()) == ['All', 'Female', 'Male']) &
               (sorted(df.loc[df['Count'] == 'Adm', 'Gender'].tolist()) == ['All', 'Female', 'Male'])

)

In [ ]:
gender_filtered

In [ ]:
df = pd.pivot_table(gender_filtered,
               index = 'Calculation1', 
               values = 'Pivot Field Values', 
               columns = ['Count', 'Gender'])[['App', 'Adm']]

In [ ]:
df

In [ ]:
df['AppMaleProportion'] = df[('App', 'Male')] / df[('App', 'All')]
df['AdmMaleProportion'] = df[('Adm', 'Male')] / df[('Adm', 'All')]

In [ ]:
# https://stackoverflow.com/questions/14507794/pandas-how-to-flatten-a-hierarchical-index-in-columns
df.columns = [''.join(col).strip() for col in df.columns.values]

In [ ]:
df

In [ ]:
# getting whether or not the school is in state

In [ ]:
# international
gender_filtered.loc[gender_filtered['County/State/ Territory'].isnull(), 'Location'] = 'INT'
gender_filtered.loc[gender_filtered['County/State/ Territory'].str.len() == 2, 'Location'] = 'OOS'
gender_filtered.loc[gender_filtered['County/State/ Territory'].str.len() > 2, 'Location'] = 'INS'

In [ ]:
df = df.merge(gender_filtered[['Calculation1', 'Location']].groupby('Calculation1').agg(lambda x: x.iloc[0])
         , on = 'Calculation1')

In [ ]:
df

In [ ]:
# now we need to repeat the above filtering for ethnicity, join the two, and then start classifying stuff
# for simplicity, we just need the 'white' proportion

In [ ]:
eth

In [ ]:
eth_filtered = eth.groupby('Calculation1').filter(lambda df: len(df.loc[df['Count'] == 'Adm', 'Count']) > 0)
eth_filtered = eth_filtered.groupby('Calculation1').filter(
    lambda df: ('White' in df.loc[df['Count'] == 'App', 'Ethnicity'].tolist()) &
               ('All' in df.loc[df['Count'] == 'App', 'Ethnicity'].tolist()) &
               ('White' in df.loc[df['Count'] == 'Adm', 'Ethnicity'].tolist()) &
               ('All' in df.loc[df['Count'] == 'Adm', 'Ethnicity'].tolist())
)

eth_filtered.loc[eth_filtered['County/State/ Territory'].isnull(), 'Location'] = 'INT'
eth_filtered.loc[eth_filtered['County/State/ Territory'].str.len() == 2, 'Location'] = 'OOS'
eth_filtered.loc[eth_filtered['County/State/ Territory'].str.len() > 2, 'Location'] = 'INS'

df_eth = pd.pivot_table(eth_filtered,
               index = 'Calculation1', 
               values = 'Pivot Field Values', 
               columns = ['Count', 'Ethnicity'])[['App', 'Adm']]

df_eth['AppWhiteProportion'] = df_eth[('App', 'White')] / df_eth[('App', 'All')]
df_eth['AdmWhiteProportion'] = df_eth[('Adm', 'White')] / df_eth[('Adm', 'All')]

df_eth.columns = [''.join(col).strip() for col in df_eth.columns.values]

df_eth = df_eth[['AppWhiteProportion', 'AdmWhiteProportion']]

df_eth = df_eth.merge(eth_filtered[['Calculation1', 'Location']].groupby('Calculation1').agg(lambda x: x.iloc[0])
         , on = 'Calculation1')

In [ ]:
# turns out no schools both reported white/all proportions AND male/female/all proportions, so these analyses need to be done separately

In [ ]:
df

### actual machine learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df.head()

In [ ]:
df['binary target'] = 1 * (df['Location'] == 'INS')

In [ ]:
df['Location'].value_counts()

In [ ]:
df['binary target'].value_counts()

In [ ]:
train_binary, test_binary = train_test_split(df, test_size = 0.2, random_state = 42)

In [ ]:
binary_models = {
    'lr': LogisticRegression(),
    'lrb': LogisticRegression(class_weight = "balanced"),
    'dt': DecisionTreeClassifier(),
    'rf': RandomForestClassifier()
}

In [ ]:
for m in binary_models:
    binary_models[m].fit(train_binary[['AppMaleProportion', 'AdmMaleProportion']], train_binary['binary target'])
    train_acc = binary_models[m].score(train_binary[['AppMaleProportion', 'AdmMaleProportion']], train_binary['binary target'])
    test_acc = binary_models[m].score(test_binary[['AppMaleProportion', 'AdmMaleProportion']], test_binary['binary target'])
    print(m, train_acc, test_acc)
    
    sns_cmap = ListedColormap(np.array(sns.color_palette())[0:2, :])

    xx, yy = np.meshgrid(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01))
    Z_string = binary_models[m].predict(np.c_[xx.ravel(), yy.ravel()])
    categories, Z_int = np.unique(Z_string, return_inverse = True)
    Z_int = Z_int.reshape(xx.shape)
    print(categories)
    plt.contourf(xx, yy, Z_int, cmap = sns_cmap)
    sns.scatterplot(data = train_binary, x = 'AppMaleProportion', y = 'AdmMaleProportion', hue = 'binary target', cmap = sns_cmap)
#     plt.title('Logistic Regression on nba_train');
#     break
    plt.show()
    

In [ ]:
binary_models['dt'].predict(train_binary[['AppMaleProportion', 'AdmMaleProportion']])

In [ ]:
# merged data

In [ ]:
df_merged = df.merge(df_eth, left_index = True, right_index = True)[['AppMaleProportion', 'AdmMaleProportion',
                                                       'AppWhiteProportion', 'AdmWhiteProportion',
                                                       'Location_x', 'binary target']].rename(
                                                        columns = {'Location_x': 'Location'})
df_merged

In [ ]:
df_merged.corr()

In [ ]:
train_merged, test_merged = train_test_split(df_merged, test_size = 0.2, random_state = 6)

In [ ]:
binary_models_merged = {
    'lr': LogisticRegression(),
    'lrb': LogisticRegression(class_weight = "balanced"),
    'dt': DecisionTreeClassifier(),
    'rf': RandomForestClassifier()
}

In [ ]:
features = ['AppMaleProportion', 'AdmMaleProportion', 'AppWhiteProportion', 'AdmWhiteProportion']

In [ ]:
for m in binary_models_merged:
    binary_models_merged[m].fit(train_merged[features], train_merged['binary target'])
    train_acc = binary_models_merged[m].score(train_merged[features], train_merged['binary target'])
    test_acc = binary_models_merged[m].score(test_merged[features], test_merged['binary target'])
    print(m, train_acc, test_acc)
    
    # could add in precision and recall
    

In [ ]:
# roc curve of binary logistic regression model

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, threshold = roc_curve(train_merged['binary target'], 
                               binary_models_merged['lr'].predict_proba(train_merged[features])[:, 1])

In [ ]:
plt.plot(fpr, tpr);
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve for unbalanced logistic regression model');

In [ ]:
fpr, tpr, threshold = roc_curve(train_merged['binary target'], 
                               binary_models_merged['lrb'].predict_proba(train_merged[features])[:, 1])

In [ ]:
plt.plot(fpr, tpr);
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve for balanced logistic regression model');

### fairness criterion

In [ ]:
Y = train_merged['binary target']
A = train_merged[features]
R = binary_models_merged['lr'].predict_proba(train_merged[features])[:, 1]

In [ ]:
# clearly these are garbage, pretty close to random model (0.5)
# only classifying well because of the severe class imbalance

In [ ]:
# plan: make A the category for whether or not a majority of applicants were male (or accepted)
# then look at separation and sufficiency

In [ ]:
A = train_merged['AppMaleProportion'] >= 0.5

In [ ]:
plt.scatter(R[A], Y[A], label = 'male majority');
plt.scatter(R[~A], Y[~A], label = 'female majority');
plt.legend();

In [ ]:
# out of state schools have more female applicants??? 

## Regression models

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# given all other features, predict AdmWhiteProportion

In [ ]:
df_ohe = pd.get_dummies(df_merged, columns = ['Location']) \
.drop(['binary target', 'Location_INS'], axis = 1) \
.rename(columns = {'AdmWhiteProportion': 'TargetProportion'}) \
.iloc[:, [0, 1, 2, 4, 5, 3]]

In [ ]:
df_ohe

In [ ]:
train_ohe, test_ohe = train_test_split(df_ohe, test_size = 0.2, random_state = 17)

In [ ]:
regression_models = {
    'ols': LinearRegression(),
    'ridge': Ridge(),
    'ridgecv': RidgeCV(),
    'lasso': Lasso(),
    'lassocv': LassoCV(),
    'knn': KNeighborsRegressor()
}

In [ ]:
for m in regression_models:
    regression_models[m].fit(train_ohe.iloc[:, :-1], train_ohe.iloc[:, -1])
    train_rmse = np.mean((regression_models[m].predict(train_ohe.iloc[:, :-1]) - train_ohe.iloc[:, -1])**2)
    test_rmse = np.mean((regression_models[m].predict(test_ohe.iloc[:, :-1]) - test_ohe.iloc[:, -1])**2)
    if m == 'knn':
        print(m, train_rmse, test_rmse)
    else:
        print(f"{m}, {np.round(train_rmse, 3)}, {np.round(test_rmse, 3)}, {np.round(regression_models[m].coef_, 3)}, {np.round(regression_models[m].intercept_, 3)}")
    

In [ ]:
train_ohe.iloc[:, -1].mean()

In [ ]:
regression_models['lasso'].predict(train_ohe.iloc[:, :-1])

In [ ]:
df_ohe

In [ ]:
train_merged